# Importacion de librerias

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import tensorflow as tf
import os

# Configuraciones de TensorFlow

In [2]:
#Definimos esta variable para no usar la GPU si llega a estar ocupada
os.environ["CUDA_VISIBLE_DEVICES"]="-1"

In [3]:
tf.enable_eager_execution()

#Comprobar que estoy ejecutandome en modo eagerly
tf.executing_eagerly()

True

In [4]:
#Log device placement corresponde a si yo quiero ver la informacion en donde se mapea o guardan las variables que creo
config = tf.ConfigProto(log_device_placement=True)

# Carga de Datos

In [ ]:
tam_img = 

# Red Neuronal